In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import interact, widgets

# Load the beer dataset 
df = pd.read_csv('beer_profile_and_ratings.csv')

# Select the columns needed for recommendation (beer name, and flavor attributes)
df = df[['Beer Name (Full)', 'Style', 'Astringency', 'Body','Alcohol', 'Bitter','Sweet', 'Sour', 'Salty', 'Fruits','Hoppy', 'Spices','Malty' ]]  # Include all relevant flavor attributes
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Beer Name (Full)  3197 non-null   object
 1   Style             3197 non-null   object
 2   Astringency       3197 non-null   int64 
 3   Body              3197 non-null   int64 
 4   Alcohol           3197 non-null   int64 
 5   Bitter            3197 non-null   int64 
 6   Sweet             3197 non-null   int64 
 7   Sour              3197 non-null   int64 
 8   Salty             3197 non-null   int64 
 9   Fruits            3197 non-null   int64 
 10  Hoppy             3197 non-null   int64 
 11  Spices            3197 non-null   int64 
 12  Malty             3197 non-null   int64 
dtypes: int64(11), object(2)
memory usage: 324.8+ KB


In [5]:
# Preprocess the data (fill any missing values)
df['Style'] = df['Style'].fillna('Unknown')
df[['Astringency', 'Body','Alcohol', 'Bitter','Sweet', 'Sour', 'Salty', 'Fruits','Hoppy', 'Spices','Malty']] = df[['Astringency', 'Body','Alcohol', 'Bitter','Sweet', 'Sour', 'Salty', 'Fruits','Hoppy', 'Spices','Malty']].fillna(0)
df

,Beer Name (Full),Style,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Alaskan Brewing Co. Alaskan Amber,Altbier,13,32,9,47,74,33,0,33,57,8,111
1,Long Trail Brewing Co. Double Bag,Altbier,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Brewing Co. Long Trail Ale,Altbier,14,37,6,42,43,11,0,10,54,4,62
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Altbier,13,55,31,47,101,18,1,49,40,16,119
4,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Altbier,25,51,26,44,45,9,1,11,51,20,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,Cisco Brewers Inc. Winter Shredder,Winter Warmer,15,37,24,35,46,19,0,23,45,67,74
3193,RJ Rockers Brewing Company First Snow Ale,Winter Warmer,15,31,23,16,54,43,0,54,14,140,58
3194,Natty Greene's Pub & Brewing Co. Red Nose Wint...,Winter Warmer,8,44,24,19,52,21,0,26,21,96,77
3195,Fish Brewing Company / Fishbowl Brewpub Fish T...,Winter Warmer,11,36,50,70,72,59,0,81,110,18,73


In [4]:
# Create sliders to select flavor weights
flavor_weights = {}
for attr in df.columns[2:]:
    flavor_weights[attr] = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description=f'{attr}:')

# Display interactive widgets for user input
@interact(**flavor_weights)
def recommend_beers(**kwargs):
    # Get user preferences for flavor attributes
    user_flavor_preferences = {attr: float(kwargs[attr]) for attr in kwargs}

    # Normalize the flavor attributes and user preferences
    scaler = MinMaxScaler()
    flavor_attributes = df[df.columns[2:]]  # All columns from 3rd column onward are assumed to be flavor attributes
    scaled_flavor_attributes = scaler.fit_transform(flavor_attributes)
    scaled_user_preferences = np.array([user_flavor_preferences[attr] for attr in flavor_attributes.columns]).reshape(1, -1)

    # Calculate flavor similarity between beers and user preferences
    flavor_similarity = cosine_similarity(scaled_flavor_attributes, scaled_user_preferences)

    # Get the indices of top 10 beers with the highest flavor similarity
    top_beer_indices = np.argsort(flavor_similarity, axis=0)[-10:][::-1]

    # Get the unique beer names and styles among the top recommendations
    unique_recommendations = df.loc[top_beer_indices.flatten()][['Beer Name (Full)','Style']].drop_duplicates()

    # Print the recommended beers
    print("Top 10 Unique Recommended Beers:")
    print(unique_recommendations)

interactive(children=(FloatSlider(value=0.5, description='Astringency:', max=1.0), FloatSlider(value=0.5, desc…